# 脚本编程与系统管理

## 通过重定向/管道/文件接受输入

In [1]:
import fileinput

# 脚本使用时就直接用管道即可
with fileinput.input('data/somefile.txt') as f:
    for line in f:
        print(line, end='')

hello world



##  终止程序并给出错误信息

In [2]:

raise SystemExit('It failed!')

SystemExit: It failed!

In [3]:
import sys

sys.stderr.write('It failed!\n')
raise SystemExit(1)

It failed!


SystemExit: 1

## 解析命令行选项

In [4]:
import argparse

parser = argparse.ArgumentParser(description='Search some files')
parser.add_argument(dest='filenames',metavar='filename', nargs='*')
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')
parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')
parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

parser.print_help()

usage: ipykernel_launcher.py [-h] -p pattern [-v] [-o OUTFILE]
                             [--speed {fast,slow}]
                             [filename [filename ...]]

Search some files

positional arguments:
  filename

optional arguments:
  -h, --help            show this help message and exit
  -p pattern, --pat pattern
                        text pattern to search for
  -v                    verbose mode
  -o OUTFILE            output file
  --speed {fast,slow}   search speed


## 运行时弹出密码输入提示

In [5]:
import getpass

user = getpass.getuser()
passwd = getpass.getpass()

def check(user, passwd):
    return passwd == 'admin'

if check(user, passwd):
    print('OK!')
else:
    print('Boo!')

OK!


## 获取终端的大小

In [6]:
import os

# print(os.get_terminal_size())
# print(os.get_terminal_size().columns)
# print(os.get_terminal_size().lines)

## 执行外部命令并获取它的输出

In [7]:
import subprocess

try:
    out_bytes = subprocess.check_output(['netstat', '-a'], timeout=5)
except subprocess.TimeoutExpired as e:
    print("error")

error


## 复制或者移动文件和目录

In [8]:
import shutil

# # Copy src to dst. (cp src dst)
# shutil.copy(src, dst)
# # Copy files, but preserve metadata (cp -p src dst)
# shutil.copy2(src, dst)
# # Copy directory tree (cp -R src dst)
# shutil.copytree(src, dst)
# # Move src to dst (mv src dst)
# shutil.move(src, dst)

## 创建和解压归档文件

In [9]:
import shutil

# shutil.unpack_archive()
# shutil.make_archive()

## 通过文件名查找文件

In [10]:
import os
def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        if name in files:
            print(relpath)

findfile('./', 'somefile.txt')

./data


## 读取配置文件

In [11]:
from configparser import ConfigParser

cfg = ConfigParser()
cfg.read('data/conf.ini')
cfg.set('server', 'port', '9000')
print(cfg.sections())
print(cfg.getint('server','port'))
print(cfg.getboolean('debug','log_errors'))

['installation', 'debug', 'server']
9000
True


In [12]:
import sys
cfg.write(sys.stdout)

[installation]
library = %(prefix)s/lib
include = %(prefix)s/include
bin = %(prefix)s/bin
prefix = /usr/local

[debug]
log_errors = true
show_warnings = False

[server]
port = 9000
nworkers = 32
pid-file = /tmp/spam.pid
root = /www/root
signature = 
	C          H         U          N



## 给简单脚本增加日志功能

In [13]:
import logging

logging.basicConfig(
    filename='data/app.log',
    level=logging.ERROR,
    format='%(levelname)s:%(asctime)s:%(message)s'
)

hostname = 'www.python.org'
item = 'spam'
filename = 'data.csv'
mode = 'r'

logging.critical('Host %s unknown', hostname)
logging.error("Couldn't find %r", item)
logging.warning('Feature is deprecated')
logging.info('Opening file %r, mode=%r', filename, mode)
logging.debug('Got here')

## 给函数库增加日志功能

In [14]:
import logging

log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

def func():
    log.critical('A Critical Error!')
    log.debug('A debug message')

## 实现一个计时器

In [15]:
import time
class Timer:
    def __init__(self, func=time.perf_counter):
        self.elapsed = 0.0
        self._func = func
        self._start = None
    
    def start(self):
        if self._start is not None:
            raise RuntimeError('Already started')
        self._start = self._func()
    
    def stop(self):
        if self._start is None:
            raise RuntimeError('Not started')
        end = self._func()
        self.elapsed += end - self._start
        self._start = None
    
    def reset(self):
        self.elapsed = 0.0
    
    @property
    def running(self):
        return self._start is not None
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args):
        self.stop()

In [16]:
def countdown(n):
    while n > 0:
        n -= 1

t = Timer()
t.start()
countdown(100000)
t.stop()
print(t.elapsed)

with Timer() as t2:
    countdown(100000)
print(t2.elapsed)

0.015008200000000471
0.015443499999999943


## 限制内存和 CPU 的使用量

In [17]:
# import signal
# import resource
# import os
# def time_exceeded(signo, frame):
#     print("Time's up!")
#     raise SystemExit(1)

# def set_max_runtime(seconds):
#     soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
#     resource.setrlimit(resource.RLIMIT_CPU, (seconds, hard))
#     signal.signal(signal.SIGXCPU, time_exceeded)

# set_max_runtime(5)
# while True:
#     pass

## 启动一个 WEB 浏览器

In [18]:
import webbrowser

webbrowser.open('http://www.python.org')

True